##### Importing Libraries

In [1]:
import Gmsh: gmsh 
using GR 

using LinearAlgebra 
using SparseArrays 
using StaticArrays
using StaticRanges

using BenchmarkTools

using Test 

using Plots 

##### Workflow

In [2]:
# Steps to develop the code

# 1. Define the structures for the points, elements and Mesh
# 2. Define the auxiliary functions required in the code
    # 2.1 Calculate transformatin points
    # 2.2 Calculate the aij(Aloc element)
    # 2.3 Calculate the Mij(Mloc element)
    # 2.4 Calculate the fij(f element)
# 3. Read the mesh file and define the mesh structures
# 4. FEM matrices assembly
    # 4.1 A matrix: Stiffness matrix
    # 4.2 M matrix: Mass matrix
    # 4.3 f vector: Load vector
# 5. Check the type stability and number of allocations with varying mesh sizes
    # Reasoning: We do this to check if the assembly of the matrices is efficient and type stable
# 6. Solving
    # 6.1 Handle the boundary conditions
    # 6.2 Solve the system of equations

##### Step 0: Generating Mesh files

In [ ]:
function generate_square_mesh(lc::Float64)
    # Initialize Gmsh
    lc = 0.1;
    gmsh.initialize()
    
    # Set Gmsh Options for Terminal Output
    gmsh.option.setNumber("General.Terminal", 1)
    
    # Add a Model Named "Square"
    gmsh.model.add("square")
    
    # Define Four Corner Points of the Square with (x, y, z) Coordinates
    gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
    gmsh.model.geo.addPoint(1, 0, 0, lc, 2)
    gmsh.model.geo.addPoint(1, 1, 0, lc, 3)
    gmsh.model.geo.addPoint(0, 1, 0, lc, 4)
    
    # Define Four Edges by Connecting Points Pairwise
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(2, 3, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 1, 4)
    
    # Define a Curved Loop by Connecting the Four Edge Labels
    gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
    
    # Define a Surface Enclosed by the Curved Loop
    gmsh.model.geo.addPlaneSurface([1], 1)
    
    # Label the Surface for Identification
    gmsh.model.setPhysicalName(2, 1, "My surface")
    
    # Synchronize the Geometric Model with the Mesh Module
    gmsh.model.geo.synchronize()
    
    # Configuration for 2nd Order Elements
    gmsh.option.setNumber("Mesh.ElementOrder", 2)
    gmsh.option.setNumber("Mesh.SecondOrderLinear", 1)
    
    # Generate Two-Dimensional Mesh
    gmsh.model.mesh.generate(2)
    
    # Dynamic filename based on 1/lc
    filename = "square_$(Int64(round(1/lc, digits=2))).msh"
    
    # Write Mesh to File
    gmsh.write(filename)
    
    # Visualize Mesh in Gmsh's GUI (if desired)
    # gmsh.fltk.run()
    
    # Finalize Gmsh to Clean Up Resources
    gmsh.finalize()
    end
    
# Example usage
# generate_square_mesh(0.1)

## I ended up using the other notebook to generate the mesh files
## ...because the code in this notebook got stuck in a loop for 
## ...reasons unknown to me.

In [2]:
lc = 0.5
filename = "square_$(Int64(round(1/lc, digits=2))).msh"
println("Mesh file written to: $filename")

Mesh file written to: square_2.msh


##### Step 1: Defining the structures

In [3]:
# struct to hold 2D point
struct Point
    x::Float64   # x coordinates
    y::Float64   # y coordinates 
  end
  
  # struct to hold a single mesh element
  struct Element
    p1::Point       # coordinates first node 
    p2::Point       # coordinates second node 
    p3::Point       # coordinates third node   
    p4::Point       # coordinates node between first and second node
    p5::Point       # coordinates node between second and third node
    p6::Point       # coordinates node between third and first node

    e1::Int64       # global index first node
    e2::Int64       # global index second node
    e3::Int64       # global index third node
    e4::Int64       # global index fourth node
    e5::Int64       # global index fifth node
    e6::Int64       # global index sixth node
  end
  
  # struct to hold entire mesh
  struct Mesh
    nnodes::Int64               # number of nodes 
    nelems::Int64               # number of elements
    Elements::Array{Element,1}  # list of Elements 
    bndNodeIds::Vector{Int64}   # indices of nodes where Dirichlet bc are applied  
    dofPerElem::Int64           # number of dofs per element 
  end

##### Step 2: Defining auxiliary functions

In [16]:
# Parameters for 3 point Gaussian quadrature
xi_gaussian = [2/3, 1/6, 1/6];
eta_gaussian = [1/6, 2/3, 1/6];
weight_gaussian = [1/3, 1/3, 1/3];

# Function to calculate the transformation coordinates
function calculate_tranformation_coordinates(xi, eta, p1, p2, p3)
    x = p1.x + (p2.x - p1.x)*xi + (p3.x - p1.x)*eta
    y = p1.y + (p2.y - p1.y)*xi + (p3.y - p1.y)*eta
    return x, y
end

# Function to calculate the aij(Aloc element)
function cfvA(x, y, a_i, c_i, d_i, b_i, e_i, a_j, c_j, d_j, b_j, e_j)
    term1 = (2*a_i*x + c_i*y + d_i) * (2*a_j*x + c_j*y + d_j)
    term2 = (2*b_i*y + c_i*x + e_i) * (2*b_j*y + c_j*x + e_j)
return term1 + term2
end

# Function to calculate the Mij(Mloc element)
function calculate_function_value_floc(x, y, a_i, b_i, c_i, d_i, e_i, f_i)
    term = (x + y) * (a_i*x^2 + b_i*y^2 + c_i*x*y + d_i*x + e_i*y + f_i);
    return term
end 

calculate_function_value_floc (generic function with 1 method)

##### Step 3: Reading mesh file and defining the structures

In [5]:
# read elements from mesh file 
function meshFromGmsh(meshFile)    
    
    #Initialize GMSH
    gmsh.initialize()
    
    #Read mesh from file
    gmsh.open(meshFile)

    # Get the mesh nodes
    # Observe that although the mesh is two-dimensional,
    # the z-coordinate that is equal to zero is stored as well.
    # Observe that the coordinates are stored contiguously for computational efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    # sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]

    # Get the mesh elements
    # Observe that we get all the two-dimensional triangular elements from the mesh
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(2)
    nelems = length(element_ids[1])
      
    # Construct uninitialized array of length nelements  
    Elements = Array{Element}(undef,nelems)

    # Construct the array of elements 
    for element_id in 1:nelems
        e1 = element_connectivity[1][6*(element_id-1)+1]
        e2 = element_connectivity[1][6*(element_id-1)+2]
        e3 = element_connectivity[1][6*(element_id-1)+3]
        e4 = element_connectivity[1][6*(element_id-1)+4]
        e5 = element_connectivity[1][6*(element_id-1)+5]
        e6 = element_connectivity[1][6*(element_id-1)+6]

        p1 = Point(sorted[e1,2], sorted[e1,3])
        p2 = Point(sorted[e2,2], sorted[e2,3])
        p3 = Point(sorted[e3,2], sorted[e3,3])
        p4 = Point(sorted[e4,2], sorted[e4,3])
        p5 = Point(sorted[e5,2], sorted[e5,3])
        p6 = Point(sorted[e6,2], sorted[e6,3])

        Elements[element_id] = Element(p1,p2,p3,p4,p5,p6,e1,e2,e3,e4,e5,e6)
    end

    # retrieve boundary nodes by loop over corner point and boundary edges
    node_ids1=[]; node_ids2=[]; node_ids3=[]; node_ids4=[]; 
    node_ids5=[]; node_ids6=[]; node_ids7=[]; node_ids8=[]; 
    node_ids1, node_coord, _ = gmsh.model.mesh.getNodes(0,1)
    node_ids2, node_coord, _ = gmsh.model.mesh.getNodes(0,2)
    node_ids3, node_coord, _ = gmsh.model.mesh.getNodes(0,3)
    node_ids4, node_coord, _ = gmsh.model.mesh.getNodes(0,4)
    node_ids5, node_coord, _ = gmsh.model.mesh.getNodes(1,1)
    node_ids6, node_coord, _ = gmsh.model.mesh.getNodes(1,2)
    node_ids7, node_coord, _ = gmsh.model.mesh.getNodes(1,3)
    node_ids8, node_coord, _ = gmsh.model.mesh.getNodes(1,4)
    bnd_node_ids = union(node_ids1,node_ids2,node_ids3,node_ids4,node_ids5,node_ids6,node_ids7,node_ids8)
    
    # Set DOF per element
    dofPerElement = 36
    
    # Store data inside mesh struct  
    mesh = Mesh(nnodes,nelems,Elements,bnd_node_ids,dofPerElement) 
    
    # Finalize gmsh
    gmsh.finalize()
    
    return mesh 
end

# read nodes from mesh file (useful for post-processing)
function nodesFromGmsh(meshFile)
    
    # Initialize GMSH
    gmsh.initialize()
    
    # Read mesh from file
    gmsh.open(meshFile)

    # Get the mesh nodes
    # Observe that although the mesh is two-dimensional,
    # the z-coordinate that is equal to zero is stored as well.
    # Observe that the coordinates are stored contiguously for computational
    # efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    # sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end]];
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]

    # Finalize gmsh
    gmsh.finalize()
    
    return xnode,ynode 
end

nodesFromGmsh (generic function with 1 method)

In [6]:
mesh = meshFromGmsh("square_1.msh")
xnode, ynode = nodesFromGmsh("square_1.msh")
display(mesh)

Mesh(13, 4, Element[Element(Point(0.0, 0.0), Point(1.0, 0.0), Point(0.5, 0.5), Point(0.5, 0.0), Point(0.75, 0.25), Point(0.25, 0.25), 1, 2, 9, 5, 10, 11), Element(Point(0.0, 1.0), Point(0.0, 0.0), Point(0.5, 0.5), Point(0.0, 0.5), Point(0.25, 0.25), Point(0.25, 0.75), 4, 1, 9, 8, 11, 12), Element(Point(1.0, 0.0), Point(1.0, 1.0), Point(0.5, 0.5), Point(1.0, 0.5), Point(0.75, 0.75), Point(0.75, 0.25), 2, 3, 9, 6, 13, 10), Element(Point(1.0, 1.0), Point(0.0, 1.0), Point(0.5, 0.5), Point(0.5, 1.0), Point(0.25, 0.75), Point(0.75, 0.75), 3, 4, 9, 7, 12, 13)], [1, 2, 3, 4, 5, 6, 7, 8], 36)

Info    : Reading 'square_1.msh'...
Info    : 9 entities
Info    : 13 nodes
Info    : 12 elements
Info    : Done reading 'square_1.msh'
Info    : Reading 'square_1.msh'...
Info    : 9 entities
Info    : 13 nodes
Info    : 12 elements
Info    : Done reading 'square_1.msh'


##### Step 4: FEM matrices assembly

In [21]:
# Function to assemble the stiffness matrix
function genLocStiffMat(element::Element, xi_gaussian, eta_gaussian, weight_gaussian)
    p1 = element.p1; p2 = element.p2; p3 = element.p3; p4 = element.p4; p5 = element.p5; p6 = element.p6;
    e1 = element.e1; e2 = element.e2; e3 = element.e3; e4 = element.e4; e5 = element.e5; e6 = element.e6;

    Iloc = SVector(e1, e1, e1, e1, e1, e1, e2, e2, e2, e2, e2, e2, e3, e3, e3, e3, e3, e3, e4, e4, e4, e4, e4, e4, e5, e5, e5, e5, e5, e5, e6, e6, e6, e6, e6, e6)
    Jloc = SVector(e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6, e1, e2, e3, e4, e5, e6)
    
    Xmat = SMatrix{6,6}(p1.x.^2, p2.x.^2, p3.x.^2, p4.x.^2, p5.x.^2, p6.x.^2,
                       p1.y.^2, p2.y.^2, p3.y.^2, p4.y.^2, p5.y.^2, p6.y.^2, 
                       p1.x*p1.y, p2.x*p2.y, p3.x*p3.y, p4.x*p4.y, p5.x*p5.y, p6.x*p6.y,
                       p1.x, p2.x, p3.x, p4.x, p5.x, p6.x, 
                       p1.y, p2.y, p3.y, p4.y, p5.y, p6.y, 
                       1, 1, 1, 1, 1, 1)
    rhs = SMatrix{6,6}(1., 0., 0., 0., 0., 0.,
                       0., 1., 0., 0., 0., 0.,
                       0., 0., 1., 0., 0., 0.,
                       0., 0., 0., 1., 0., 0.,
                       0., 0., 0., 0., 1., 0.,
                       0., 0., 0., 0., 0., 1.)
    Emat = SMatrix{6,6}(Xmat\rhs)

    x_quad1, y_quad1 = calculate_tranformation_coordinates(xi_gaussian[1], eta_gaussian[1], p1, p2, p3);
    x_quad2, y_quad2 = calculate_tranformation_coordinates(xi_gaussian[2], eta_gaussian[2], p1, p2, p3);
    x_quad3, y_quad3 = calculate_tranformation_coordinates(xi_gaussian[3], eta_gaussian[3], p1, p2, p3);

    fp1 = SMatrix{6,6}(cfvA(x_quad1, y_quad1, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]),
                       cfvA(x_quad1, y_quad1, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]),
                       cfvA(x_quad1, y_quad1, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]),
                       cfvA(x_quad1, y_quad1, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]),
                       cfvA(x_quad1, y_quad1, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]),
                       cfvA(x_quad1, y_quad1, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]));

    fp2 = SMatrix{6,6}(cfvA(x_quad2, y_quad2, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]),
                       cfvA(x_quad2, y_quad2, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]),
                       cfvA(x_quad2, y_quad2, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]),
                       cfvA(x_quad2, y_quad2, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]),
                       cfvA(x_quad2, y_quad2, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]),
                       cfvA(x_quad2, y_quad2, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]));
    
    fp3 = SMatrix{6,6}(cfvA(x_quad3, y_quad3, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1]),
                       cfvA(x_quad3, y_quad3, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2]),
                       cfvA(x_quad3, y_quad3, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3]),
                       cfvA(x_quad3, y_quad3, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4]),
                       cfvA(x_quad3, y_quad3, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5]),
                       cfvA(x_quad3, y_quad3, Emat[1,1], Emat[3,1], Emat[4,1], Emat[2,1], Emat[5,1], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,2], Emat[3,2], Emat[4,2], Emat[2,2], Emat[5,2], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,3], Emat[3,3], Emat[4,3], Emat[2,3], Emat[5,3], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,4], Emat[3,4], Emat[4,4], Emat[2,4], Emat[5,4], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,5], Emat[3,5], Emat[4,5], Emat[2,5], Emat[5,5], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]), cfvA(x_quad1, y_quad1, Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6], Emat[1,6], Emat[3,6], Emat[4,6], Emat[2,6], Emat[5,6]));

    jacobian_factor = (p2.x - p1.x)*(p3.y - p1.y) - (p3.x - p1.x)*(p2.y - p1.y);

    Amat = SMatrix{6,6}(0.5*jacobian_factor*(weight_gaussian[1]*fp1 + weight_gaussian[2]*fp2 + weight_gaussian[3]*fp3));
    # for i in 1:6
    #     for j in 1:6
    #         fp1 = cfvA(x_quad1, y_quad1, Emat[1,i], Emat[3,i], Emat[4,i], Emat[2,i], Emat[5,i], Emat[1,j], Emat[3,j], Emat[4,j], Emat[2,j], Emat[5,j]);
    #         fp2 = cfvA(x_quad2, y_quad2, Emat[1,i], Emat[3,i], Emat[4,i], Emat[2,i], Emat[5,i], Emat[1,j], Emat[3,j], Emat[4,j], Emat[2,j], Emat[5,j]);
    #         fp3 = cfvA(x_quad3, y_quad3, Emat[1,i], Emat[3,i], Emat[4,i], Emat[2,i], Emat[5,i], Emat[1,j], Emat[3,j], Emat[4,j], Emat[2,j], Emat[5,j]);
    #         Amat[i,j] = (1/2)*jacobian_factor*(weight_gaussian[1]*fp1 + weight_gaussian[2]*fp2 + weight_gaussian[3]*fp3);
    #     end
    # end

    Aloc = [Amat[1,:]; Amat[2,:]; Amat[3,:]; Amat[4,:]; Amat[5,:]; Amat[6,:]];
    return Iloc, Jloc, Aloc
end

function genStiffMat(mesh::Mesh)

    nelems = mesh.nelems
    dofPerElem = mesh.dofPerElem

    Avalues = zeros(Float64, dofPerElem*nelems)
    I = zeros(Int64, length(Avalues))
    J = zeros(Int64, length(Avalues))

    for i = 1:nelems
        element = mesh.Elements[i]
        Iloc, Jloc, Aloc = genLocStiffMat(element, xi_gaussian, eta_gaussian, weight_gaussian)
        irange = mrange(dofPerElem*i-35, dofPerElem*i)
        I[irange] = Iloc
        J[irange] = Jloc
        Avalues[irange] = Aloc
    end

    A = sparse(I,J,Avalues)

    return A;
end

genStiffMat (generic function with 1 method)

In [22]:
mesh = meshFromGmsh("square_1.msh")
A = genStiffMat(mesh);

Info    : Reading 'square_1.msh'...
Info    : 9 entities
Info    : 13 nodes
Info    : 12 elements
Info    : Done reading 'square_1.msh'


In [23]:
mesh = meshFromGmsh("square_1.msh");   @time genStiffMat(mesh); # <= force recompilation 
mesh = meshFromGmsh("square_1.msh");   @time genStiffMat(mesh);
mesh = meshFromGmsh("square_2.msh");   @time genStiffMat(mesh); 
mesh = meshFromGmsh("square_10.msh");  @time genStiffMat(mesh); 
mesh = meshFromGmsh("square_20.msh"); @time genStiffMat(mesh);
mesh = meshFromGmsh("square_100.msh"); @time genStiffMat(mesh);

Info    : Reading 'square_1.msh'...
Info    : 9 entities
Info    : 13 nodes
Info    : 12 elements
Info    : Done reading 'square_1.msh'
  0.000064 seconds (49 allocations: 19.891 KiB)


Info    : Reading 'square_1.msh'...
Info    : 9 entities
Info    : 13 nodes
Info    : 12 elements
Info    : Done reading 'square_1.msh'
  0.000046 seconds (49 allocations: 19.891 KiB)
Info    : Reading 'square_2.msh'...
Info    : 9 entities
Info    : 37 nodes
Info    : 26 elements
Info    : Done reading 'square_2.msh'
  0.000089 seconds (139 allocations: 67.125 KiB)
Info    : Reading 'square_10.msh'...
Info    : 9 entities
Info    : 525 nodes
Info    : 286 elements
Info    : Done reading 'square_10.msh'
  0.000968 seconds (2.20 k allocations: 1.105 MiB)


Info    : Reading 'square_20.msh'...
Info    : 9 entities
Info    : 1969 nodes
Info    : 1028 elements
Info    : Done reading 'square_20.msh'
  0.004333 seconds (8.52 k allocations: 4.300 MiB)
Info    : Reading 'square_100.msh'...
Info    : 9 entities
Info    : 46929 nodes
Info    : 23668 elements
Info    : Done reading 'square_100.msh'
  0.170165 seconds (209.40 k allocations: 105.819 MiB, 14.72% gc time)

In [ ]:
# Before making xi, eta and weight_gaussian as parameters and removing a to f vectors 
# The number of allocation actually increased after that which I don't understand
# And the number of allocations increased even more after I took the for loop out

# Info    : Reading 'square_1.msh'...
# Info    : 9 entities
# Info    : 13 nodes
# Info    : 12 elements
# Info    : Done reading 'square_1.msh'
#   0.000051 seconds (33 allocations: 14.703 KiB)
# Info    : Reading 'square_1.msh'...
# Info    : 9 entities
# Info    : 13 nodes
# Info    : 12 elements
# Info    : Done reading 'square_1.msh'
#   0.000049 seconds (33 allocations: 14.703 KiB)
# Info    : Reading 'square_2.msh'...
# Info    : 9 entities
# Info    : 37 nodes
# Info    : 26 elements
# Info    : Done reading 'square_2.msh'
#   0.000082 seconds (83 allocations: 48.969 KiB)
# Info    : Reading 'square_10.msh'...
# Info    : 9 entities
# Info    : 525 nodes
# Info    : 286 elements
# Info    : Done reading 'square_10.msh'
#   0.001114 seconds (1.23 k allocations: 817.469 KiB)
# Info    : Reading 'square_20.msh'...
# Info    : 9 entities
# Info    : 1969 nodes
# Info    : 1028 elements
# Info    : Done reading 'square_20.msh'
#   0.006796 seconds (4.74 k allocations: 3.105 MiB)
# Info    : Reading 'square_100.msh'...
# Info    : 9 entities
# Info    : 46929 nodes
# Info    : 23668 elements
# Info    : Done reading 'square_100.msh'
#   0.204760 seconds (116.34 k allocations: 76.355 MiB, 14.71% gc time)

In [19]:
@code_warntype genStiffMat(mesh)

MethodInstance for genStiffMat(::Mesh)
  from genStiffMat(mesh::Mesh) @ Main d:\Python Codes\FEM Jupyter Notebook\2order.ipynb:52
Arguments
  #self#::Core.Const(genStiffMat)
  mesh::Mesh
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  A::SparseMatrixCSC{Float64, Int64}
  J::Vector{Int64}
  I::Vector{Int64}
  Avalues::Vector{Float64}
  dofPerElem::Int64
  nelems::Int64
  @_10::Int64
  i::Int64
  irange::MutableRange{Int64, UnitRange{Int64}}
  Aloc::MVector{36, Float64}
  Jloc::SVector{36, Int64}
  Iloc::SVector{36, Int64}
  element::Element
Body::SparseMatrixCSC{Float64, Int64}
1 ─       Core.NewvarNode(:(A))
│         (nelems = Base.getproperty(mesh, :nelems))
│         (dofPerElem = Base.getproperty(mesh, :dofPerElem))
│   %4  = Main.Float64::Core.Const(Float64)
│   %5  = (dofPerElem * nelems)::Int64
│         (Avalues = Main.zeros(%4, %5))
│   %7  = Main.Int64::Core.Const(Int64)
│   %8  = Main.length(Avalues)::Int64
│         (I = Main.zeros(%7, %8))
│   %10 = Main.Int64::Core.Co

       (@_3 = Base.iterate(%13))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
2 ┄ %18 = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%18, 1))
│   %20 = Core.getfield(%18, 2)::Int64
│   %21 = Base.getproperty(mesh, :Elements)::Vector{Element}
│         (element = Base.getindex(%21, i))
│   %23 = Main.genLocStiffMat(element)::Tuple{SVector{36, Int64}, SVector{36, Int64}, MVector{36, Float64}}
│   %24 = Base.indexed_iterate(%23, 1)::Core.PartialStruct(Tuple{SVector{36, Int64}, Int64}, Any[SVector{36, Int64}, Core.Const(2)])
│         (Iloc = Core.getfield(%24, 1))
│         (@_10 = Core.getfield(%24, 2))
│   %27 = Base.indexed_iterate(%23, 2, @_10::Core.Const(2))::Core.PartialStruct(Tuple{SVector{36, Int64}, Int64}, Any[SVector{36, Int64}, Core.Const(3)])
│         (Jloc = Core.getfield(%27, 1))
│         (@_10 = Core.getfield(%27, 2))
│   %30 = Base.indexed_iterate(%23, 3, @_10::Core.Const(3))::Core.PartialStruct(Tuple{MVector{

%33 = (%32 - 35)::Int64
│   %34 = (dofPerElem * i)::Int64
│         (irange = Main.mrange(%33, %34))
│         Base.setindex!(I, Iloc, irange)
│         Base.setindex!(J, Jloc, irange)
│         Base.setindex!(Avalues, Aloc, irange)
│         (@_3 = Base.iterate(%13, %20))
│   %40 = (@_3 === nothing)::Bool
│   %41 = Base.not_int(%40)::Bool
└──       goto #4 if not %41
3 ─       goto #2
4 ┄       (A = Main.sparse(I, J, Avalues))
└──       return A



In [ ]:
# Function to assemble the f vector